**This is example of how to trace model with jit and export it to the onnx**

In [1]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


In [2]:
ray.init(object_store_memory=1024*1024*1000)

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RayContext(dashboard_url='', python_version='3.9.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '10.2.168.57', 'raylet_ip_address': '10.2.168.57', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-09-16_12-52-47_511930_1195699/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-09-16_12-52-47_511930_1195699/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-09-16_12-52-47_511930_1195699', 'metrics_export_port': 60979, 'gcs_address': '10.2.168.57:60125', 'address': '10.2.168.57:60125', 'node_id': 'fe6c272b6e426916a75b5fd0f7bea70df635739d1f8f323859dcbf3c'})

In [3]:
config_name = '../rl_games/configs/ppo_cartpole.yaml'

In [4]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['params']['config']['normalize_input'] = False
    config['params']['config']['max_epochs'] = 1000
    config['params']['config']['full_experiment_name'] = 'cartpole_onnx'
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

self.seed = 1663357969
Started to train
Exact experiment name requested from command line: cartpole_onnx
current training device: cpu
build mlp: 4
build mlp: 4


/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/rl_games/common/experience.py:341: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.tensor_dict['actions'] = self._create_tensor_from_space(gym.spaces.Box(low=0, high=1,shape=self.actions_shape, dtype=np.long), obs_base_shape)


fps step: 3911 fps step and policy inference: 3144 fps total: 2097 epoch: 1/1000
fps step: 3312 fps step and policy inference: 2754 fps total: 2070 epoch: 2/1000
fps step: 3295 fps step and policy inference: 2740 fps total: 2090 epoch: 3/1000
fps step: 3196 fps step and policy inference: 2669 fps total: 2053 epoch: 4/1000
fps step: 3297 fps step and policy inference: 2747 fps total: 2025 epoch: 5/1000
fps step: 3002 fps step and policy inference: 2524 fps total: 1853 epoch: 6/1000
fps step: 3206 fps step and policy inference: 2668 fps total: 1971 epoch: 7/1000
fps step: 3209 fps step and policy inference: 2664 fps total: 1955 epoch: 8/1000
fps step: 3172 fps step and policy inference: 2661 fps total: 2022 epoch: 9/1000
fps step: 3036 fps step and policy inference: 2545 fps total: 1976 epoch: 10/1000
saving next best rewards:  [24.014812]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3138 fps step and policy inference: 2651 fps total: 1945 epoch: 11/1000
s

fps step: 3244 fps step and policy inference: 2762 fps total: 2004 epoch: 56/1000
saving next best rewards:  [99.066605]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3219 fps step and policy inference: 2734 fps total: 2097 epoch: 57/1000
saving next best rewards:  [100.825935]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3158 fps step and policy inference: 2689 fps total: 2065 epoch: 58/1000
saving next best rewards:  [103.]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3319 fps step and policy inference: 2803 fps total: 2062 epoch: 59/1000
saving next best rewards:  [105.7804]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3218 fps step and policy inference: 2710 fps total: 1985 epoch: 60/1000
saving next best rewards:  [111.9443]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3293 fps step and policy inference: 2779 fps total: 2112

fps step: 3404 fps step and policy inference: 2876 fps total: 2074 epoch: 109/1000
fps step: 3186 fps step and policy inference: 2703 fps total: 2028 epoch: 110/1000
saving next best rewards:  [196.90764]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3274 fps step and policy inference: 2785 fps total: 2113 epoch: 111/1000
fps step: 3045 fps step and policy inference: 2588 fps total: 1937 epoch: 112/1000
fps step: 3328 fps step and policy inference: 2795 fps total: 2121 epoch: 113/1000
saving next best rewards:  [198.93497]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3208 fps step and policy inference: 2721 fps total: 2050 epoch: 114/1000
saving next best rewards:  [199.00563]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3156 fps step and policy inference: 2654 fps total: 1989 epoch: 115/1000
fps step: 3333 fps step and policy inference: 2814 fps total: 2127 epoch: 116/1000
fps step: 3165 fps

fps step: 3343 fps step and policy inference: 2805 fps total: 2113 epoch: 194/1000
fps step: 3096 fps step and policy inference: 2632 fps total: 1986 epoch: 195/1000
fps step: 3295 fps step and policy inference: 2767 fps total: 2046 epoch: 196/1000
fps step: 3209 fps step and policy inference: 2703 fps total: 2069 epoch: 197/1000
fps step: 3126 fps step and policy inference: 2646 fps total: 1877 epoch: 198/1000
fps step: 2269 fps step and policy inference: 1978 fps total: 1445 epoch: 199/1000
fps step: 2240 fps step and policy inference: 1913 fps total: 1458 epoch: 200/1000
fps step: 2759 fps step and policy inference: 2346 fps total: 1820 epoch: 201/1000
fps step: 3053 fps step and policy inference: 2570 fps total: 1922 epoch: 202/1000
fps step: 3041 fps step and policy inference: 2588 fps total: 1899 epoch: 203/1000
fps step: 3196 fps step and policy inference: 2701 fps total: 2010 epoch: 204/1000
fps step: 2927 fps step and policy inference: 2510 fps total: 1920 epoch: 205/1000
fps 

fps step: 3136 fps step and policy inference: 2647 fps total: 1977 epoch: 293/1000
fps step: 3064 fps step and policy inference: 2611 fps total: 1919 epoch: 294/1000
fps step: 3095 fps step and policy inference: 2639 fps total: 2037 epoch: 295/1000
fps step: 3093 fps step and policy inference: 2629 fps total: 1926 epoch: 296/1000
fps step: 3160 fps step and policy inference: 2684 fps total: 2014 epoch: 297/1000
fps step: 3074 fps step and policy inference: 2605 fps total: 1946 epoch: 298/1000
fps step: 2999 fps step and policy inference: 2532 fps total: 1967 epoch: 299/1000
fps step: 3062 fps step and policy inference: 2611 fps total: 1914 epoch: 300/1000
fps step: 3173 fps step and policy inference: 2699 fps total: 1974 epoch: 301/1000
fps step: 3325 fps step and policy inference: 2794 fps total: 2123 epoch: 302/1000
fps step: 3083 fps step and policy inference: 2622 fps total: 2021 epoch: 303/1000
fps step: 2953 fps step and policy inference: 2539 fps total: 1950 epoch: 304/1000
fps 

fps step: 3038 fps step and policy inference: 2598 fps total: 1947 epoch: 362/1000
fps step: 3099 fps step and policy inference: 2632 fps total: 1918 epoch: 363/1000
fps step: 3047 fps step and policy inference: 2575 fps total: 1978 epoch: 364/1000
fps step: 3018 fps step and policy inference: 2581 fps total: 1965 epoch: 365/1000
fps step: 3104 fps step and policy inference: 2636 fps total: 1953 epoch: 366/1000
fps step: 3116 fps step and policy inference: 2641 fps total: 1941 epoch: 367/1000
fps step: 3212 fps step and policy inference: 2728 fps total: 1986 epoch: 368/1000
saving next best rewards:  [328.3835]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3175 fps step and policy inference: 2704 fps total: 2075 epoch: 369/1000
fps step: 3087 fps step and policy inference: 2629 fps total: 1934 epoch: 370/1000
saving next best rewards:  [333.4807]
=> saving checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
fps step: 3097 fps step and policy inferenc

In [7]:
class ModelWrapper(torch.nn.Module):
    '''
    Main idea is to ignore outputs which we don't need from model
    '''
    def __init__(self, model):
        torch.nn.Module.__init__(self)
        self._model = model
        
        
    def forward(self,input_dict):
        input_dict['obs'] = self._model.norm_obs(input_dict['obs'])
        '''
        just model export doesn't work. Looks like onnx issue with torch distributions
        thats why we are exporting only neural network
        '''
        #print(input_dict)
        #output_dict = self._model.a2c_network(input_dict)
        #input_dict['is_train'] = False
        #return output_dict['logits'], output_dict['values']
        return self._model.a2c_network(input_dict)

In [8]:
agent = runner.create_player()
agent.restore('runs/cartpole_onnx/nn/cartpole_vel_info.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states,
}

with torch.no_grad():
    adapter = flatten.TracingAdapter(ModelWrapper(agent.model), inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "cartpole.onnx", verbose=True, input_names=['obs'], output_names=['logits', 'value'])

onnx_model = onnx.load("cartpole.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

{'observation_space': Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32), 'action_space': Discrete(2), 'agents': 1, 'value_size': 1}
build mlp: 4
build mlp: 4
=> loading checkpoint 'runs/cartpole_onnx/nn/cartpole_vel_info.pth'
1 1
(tensor([[0., 0., 0., 0.]], device='cuda:0'),) (tensor([[0., 0., 0., 0.]], device='cuda:0'),)
{'obs': tensor([[0., 0., 0., 0.]], device='cuda:0'), 'rnn_states': None, 'is_train': False}
(tensor([[-0.0608,  0.1237]], device='cuda:0'), tensor([[9.7718]], device='cuda:0'))
graph(%obs : Float(1, 4, strides=[4, 1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.actor_mlp.0.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.actor_mlp.0.weight : Float(32, 4, strides=[4, 1], requires_grad=0, device=cuda:0),
      %model._model.a2c_network.actor_mlp.2.bias : Float(32, strides=[1], requires_grad=0, device=cuda:0),
     

In [9]:
ort_model = ort.InferenceSession("cartpole.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 4)).astype(np.float32)},
)
print(outputs)

[array([[-0.06092914,  0.12378628]], dtype=float32), array([[9.77019]], dtype=float32)]


In [10]:
is_done = False
env = agent.env
obs = env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    action = np.argmax(outputs[0])
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    #plt.imshow(screen)
    #display.display(plt.gcf())    
    #display.clear_output(wait=True)
print(total_reward, num_steps)
#ipythondisplay.clear_output(wait=True)

500.0 500
